In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
farm_land = 200
groups = list(range(1, 5))
group_size  = dict(zip(groups, [20, 30, 20, 10]))
group_yield = dict(zip(groups, [1.1, 0.9, 0.8, 0.65]))

labour_req  = dict(zip(['heifer', 'milk_cow', 'land_grain', 'land_beet'],
                       [10, 42, 4, 14]))
other_costs = dict(zip(['heifer', 'milk_cow', 'land_grain', 'land_beet'],
                       [50, 100, 15, 10]))

years = list(range(1, 6))
ages = list(range(1, 13))



In [22]:
model = gp.Model('Farm Planning')

# add vars
x = model.addVars(groups, years,
                  name='make_grain')
y = model.addVars(years,
                  name='make_beet')
z = model.addVars(years,
                  name='buy_grain')
s = model.addVars(years,
                  name='sell_grain')
u = model.addVars(years,
                  name='buy_beet')
v = model.addVars(years,
                  name='sell_beet')
l = model.addVars(years,
                  name='xtra_labour')
m = model.addVars(years,
                  name='outlay')
n = model.addVars(years,
                  name='sell_heifer')
q = model.addVars(ages, years,
                  name='qty_cow')
r = model.addVars(years,
                  name='qty_cow0')
p = model.addVars(years,
                  name='profit')

# objective function
model.setObjective((gp.quicksum(p[t] for t in years)
                  - 39.71*gp.quicksum((4+t)*m[t] for t in years)),
                  GRB.MAXIMIZE)

# constraints

    # continuity ok
model.addConstrs((q[1,t+1] == 0.95*r[t] for t in years[:4]), # book is wrong? t=1..4
                 name='cont_a')
model.addConstrs((q[2,t+1] == 0.95*q[1,t] for t in years[:4]), # book is wrong? t=1..4
                 name='cont_b')
model.addConstrs((q[j+1,t+1] == 0.98*q[j,t] for t in years[:4] for j in ages[1:11]),
                 name='cont_c')
model.addConstrs((r[t] + n[t] == 1.1/2*gp.quicksum(q[j,t] for j in ages[1:11]) 
                  for t in years),
                 name='cont_d')

    # initial conditions ok
model.addConstrs(((q[j,1] == 9.5) for j in ages[:2]),
                 name='init_cond')
model.addConstrs(((q[j,1] == 9.8) for j in ages[2:]),
                 name='init_cond')

    # accomodation ok
model.addConstrs((r[t] + gp.quicksum(q[j,t] for j in ages[:11])
                   <= 130 + gp.quicksum(m[k] for k in years if k<=t)
                 for t in years),
                 name='accomodation')

    # grain consumption ok, mas esquisito
model.addConstrs((0.6*gp.quicksum(q[j,t] for j in ages[1:11]) - z[t] + s[t]
                   <= gp.quicksum(x[i,t] for i in groups)
                 for t in years),
                name='grain_consump')

    # beet consumption
model.addConstrs((0.7*gp.quicksum(q[j,t] for j in ages[1:11])
                   <= y[t] + u[t] - v[t] for t in years),
                name='beet_consump')

    # grain growing ok
model.addConstrs((x[1,t] <= 1.1*20 for t in years),
                 name='grain_growing')
model.addConstrs((x[2,t] <= 0.9*30 for t in years),
                 name='grain_growing')
model.addConstrs((x[3,t] <= 0.8*20 for t in years),
                 name='grain_growing')
model.addConstrs((x[4,t] <= 0.65*10 for t in years),
                 name='grain_growing')

    # acreage
model.addConstrs((1/1.1*x[1,t] + 1/0.9*x[2,t] + 1/0.8*x[3,t] + 1/0.65*x[4,t] 
                + 1/1.5*y[t] + 2/3*r[t] + 2/3*q[1,t] 
                + gp.quicksum(q[j,t] for j in ages[1:11]) 
                  <= 200 for t in years),
                 name='acreage')
    
    # labour
model.addConstrs((0.04*(1/1.1*x[1,t] + 1/0.9*x[2,t] + 1/0.8*x[3,t] + 1/0.65*x[4,t]) 
                + 0.14/1.5*y[t] + 0.1*r[t] + 0.1*q[1,t] 
                + 0.42*gp.quicksum(q[j,t] for j in ages[1:11]) 
                  <= 55 + l[t] for t in years),
                 name='labour')

    # end total ok
model.addConstr((gp.quicksum(q[j,5] for j in ages[1:11]) <= 175),
                name='end_total')
model.addConstr((gp.quicksum(q[j,5] for j in ages[1:11]) >= 50),
                name='end_total')

    # profit ok
model.addConstrs((p[t] == 30*1.1/2*gp.quicksum(q[j,t] for j in ages[1:11])
                        + 40*n[t] + 120*q[12,t] 
                        + 370*gp.quicksum(q[j,t] for j in ages[1:11])
                        + 75*s[t] + 58*v[t] - 90*z[t] - 70*u[t] - 120*l[t]
                        - 4000 - 50*r[t] - 50*q[1,t]
                        - 100*gp.quicksum(q[j,t] for j in ages[1:11])
                        - 15*(1/1.1*x[1,t] + 1/0.9*x[2,t] + 1/0.8*x[3,t] + 1/0.65*x[4,t])
                        - 10/1.5*y[t] - 39.71*gp.quicksum(m[k] for k in years if k<=t)
                  for t in years),
                 name='profit')

    # positive profit ok
model.addConstrs((p[t] >= 0 for t in years),
                 name='pos_profit')

model.update()
model.write('Farm Planning.lp')
model.Params.DualReductions = 1
model.optimize()

Parameter DualReductions unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 122 rows, 130 columns and 748 nonzeros
Model fingerprint: 0x64a79d3e
Coefficient statistics:
  Matrix range     [4e-02, 3e+02]
  Objective range  [1e+00, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 4e+03]
Presolve removed 90 rows and 66 columns
Presolve time: 0.02s
Presolved: 32 rows, 64 columns, 252 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1200000e+32   5.000000e+30   5.120000e+02      0s
      23    1.2171917e+05   0.000000e+00   0.000000e+00      0s

Solved in 23 iterations and 0.03 seconds
Optimal objective  1.217191729e+05


In [45]:
print('Total Profit of ${:.2f}'.format(model.objVal))

for t in years:
    print('Period {}'.format(t))
    print('Profit of ${:.2f}'.format(p[t].x))
    for i in groups:
        if x[i,t].x > 0:
            print('\tPlant {:.1f} acres of Grains in Land of group {}'.format(x[i,t].x, i))
    if z[t].x > 0:
        print('\tBuy {:.1f} tons of Grains'.format(z[t].x))
    if s[t].x > 0:
        print('\tSell {:.1f} tons of Grains'.format(s[t].x))
    if y[t].x > 0:
        print('\tPlant {:.1f} acres of Sugar Beet'.format(y[t].x))
    if u[t].x > 0:
        print('\tBuy {:.1f} tons of Sugar Beet'.format(u[t].x))
    if v[t].x > 0:
        print('\tSell {:.1f} tons of Sugar Beet'.format(v[t].x))
    if l[t].x > 0:
        print('\tContract {:.1f} hours of extra labour'.format(l[t].x))
    if n[t].x > 0:
        print('\tSell {:.0f} heifers'.format(n[t].x))
    if r[t].x > 0:
        print('\tRaise {:.0f} newborn cows'.format(r[t].x))

Total Profit of $121719.17
Period 1
Profit of $21906.06
	Plant 22.0 acres of Grains in Land of group 1
	Buy 36.6 tons of Grains
	Plant 91.1 acres of Sugar Beet
	Sell 22.8 tons of Sugar Beet
	Sell 31 heifers
	Raise 23 newborn cows
Period 2
Profit of $21888.70
	Plant 22.0 acres of Grains in Land of group 1
	Buy 35.1 tons of Grains
	Plant 94.0 acres of Sugar Beet
	Sell 27.4 tons of Sugar Beet
	Sell 41 heifers
	Raise 12 newborn cows
Period 3
Profit of $25816.06
	Plant 22.0 acres of Grains in Land of group 1
	Plant 2.8 acres of Grains in Land of group 2
	Buy 37.8 tons of Grains
	Plant 97.7 acres of Sugar Beet
	Sell 24.6 tons of Sugar Beet
	Sell 57 heifers
Period 4
Profit of $26825.77
	Plant 22.0 acres of Grains in Land of group 1
	Buy 40.1 tons of Grains
	Plant 114.6 acres of Sugar Beet
	Sell 42.1 tons of Sugar Beet
	Sell 57 heifers
Period 5
Profit of $25282.59
	Plant 22.0 acres of Grains in Land of group 1
	Buy 33.5 tons of Grains
	Plant 131.3 acres of Sugar Beet
	Sell 66.6 tons of Sugar B